In [12]:
def main():
    import csv
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver.support.ui import WebDriverWait
    import selenium.webdriver.support.expected_conditions as EC
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome('/home/hung/Downloads/chromedriver')
    url = "https://www.thegioididong.com/laptop#pi=8"
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    result = soup.find_all('li',{'class':'item ajaxed __cate_44'})
    scraped = 0
    for item in result:
        url0 = 'https://www.thegioididong.com' + item.a.get('href') + '#top-tskt'
        driver.get(url0)
        btn = driver.find_element(By.CSS_SELECTOR,'.btn-detail.btn-short-spec').click()
        WebDriverWait(driver,timeout=5).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.parameter-all')))
        soup0 = BeautifulSoup(driver.page_source, 'html.parser')
        all =  soup0.find('div',{'class':'content-t-wrap'}).find_all('li')
        if len(all) >= 30:
            print(len(all))
        scraped += 1
        print(scraped)
    print(len(result),"scrapped:",scraped)
# main()

In [13]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome('/home/hung/Downloads/chromedriver')

<ipython-input-13-8dd47fe60a14>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/hung/Downloads/chromedriver')


In [14]:
url = "https://www.thegioididong.com/laptop#pi=8"

driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
result = soup.find_all('li',{'class':'item ajaxed __cate_44'})
print(len(result))

171


# Prototype

In [15]:
item = result[0]
title = item.a.h3.text.strip()
print(title)

Acer Nitro 5 Gaming AN515 57 727J i7 11800H (NH.QD9SV.005.)


In [16]:
url0 = 'https://www.thegioididong.com' + item.a.get('href')
driver.get(url0)
driver.find_element_by_css_selector('.btn-detail.btn-short-spec').click()
soup0 = BeautifulSoup(driver.page_source, 'html.parser')

<ipython-input-16-3d7a30b75a15>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('.btn-detail.btn-short-spec').click()


In [17]:
spec_id_to_name = {
    '28727':'cpu_name',
    '28859':'cpu_ncores',
    '28860':'cpu_nthreads',
    '93':'cpu_basefreq',
    '97':'cpu_turbofreq',
    '28861':'cpu_cache',
    
    '146':'ram_size',
    '149':'ram_type',
    '155':'ram_clock',
    '137':'ram_upgradable',
    
    '184':'storage',
    '28123':'gpu',
    '228':'battery',
    
    '187':'displ_size',
    '189':'displ_resolution',
    '29056':'displ_rate',
    '186':'displ_tech',
    '480':'displ_touchscreen',

    '196':'sound',
    '200':'ports',
    '218':'sd_card',
    '206':'wless_connect',
    '223':'webcam',
    '201':'special',
    '10741':'backlight_key',

    '7779':'weight_dims',
    '7903':'surface',
    '8599':'os',
    '22711':'launch'
}

In [18]:
def spec_scrap(data_id,soup0):
    '''data_id: str'''
    li = soup0.find('li',{'data-id':data_id})
    if not li:
        return ''
    ls = []
    for p in li.find_all('p',{'class':'circle'}):
        if p.text: ls.append(p.text.strip())
        else: ls.append(p.a.text.strip())
    os = "|".join(ls)
    if not os:
        try:
            os = li.span.text.strip()
        except:
            os = '|'.join(a.text.strip() for a in li.find_all('a'))
    return os

In [19]:
title = soup0.find('h1').text.strip()
price = soup0.find('p',{'class':'box-price-present'}).text.strip()
rating = soup0.find('p',{'class':'point'}).text.strip()
review_count = soup0.find('a',{'class':'rating-total'}).text.strip()
cmt_count = soup0.find('span',{'class':'totalcomment'}).text[:22].strip()
print(cmt_count)

157 bình luận


# Main function

In [ ]:
headers = ['title','price','rating','review_count','cmt_count'] + list(spec_id_to_name.values())
result = pd.DataFrame(columns = headers)
scraped = 0 #track the number of scraped laptop

url = "https://www.thegioididong.com/laptop#pi=10"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
laptops = soup.find_all('li',{'class':'item ajaxed __cate_44'})

#for each row
for item in laptops:
    url0 = 'https://www.thegioididong.com' + item.a.get('href') + '#top-tskt'
    driver.get(url0)
    soup0 = BeautifulSoup(driver.page_source, 'html.parser')
    
    #start scraping title, price, rating, review_count, cmt_count
    title = soup0.find('h1').text.strip()
    try: price = soup0.find('p',{'class':'box-price-present'}).text.strip()
    except: price = ''
    try: rating = soup0.find('p',{'class':'point'}).text.strip()
    except: rating = ''
    try: review_count = soup0.find('a',{'class':'rating-total'}).text.strip()
    except: review_count = ''
    try: cmt_count = soup0.find('span',{'class':'totalcomment'}).text[:22].strip()
    except: cmt_count=''

    #click the button
    btn = driver.find_element(By.CSS_SELECTOR,'.btn-detail.btn-short-spec').click()
    WebDriverWait(driver,timeout=5).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.parameter-all')))
    
    #start scraping specs
    soup0 = BeautifulSoup(driver.page_source, 'html.parser')
    specs_row = [spec_scrap(id,soup0) for id in spec_id_to_name]
    
    #saved the result
    result.loc[len(result.index)] = [title,price,rating,review_count,cmt_count] + specs_row
    scraped += 1
    print(len(laptops),"scrapped:",scraped)

171 scrapped: 1
171 scrapped: 2
171 scrapped: 3
171 scrapped: 4
171 scrapped: 5
171 scrapped: 6
171 scrapped: 7
171 scrapped: 8
171 scrapped: 9
171 scrapped: 10
171 scrapped: 11
171 scrapped: 12
171 scrapped: 13
171 scrapped: 14
171 scrapped: 15
171 scrapped: 16
171 scrapped: 17
171 scrapped: 18
171 scrapped: 19
171 scrapped: 20
171 scrapped: 21
171 scrapped: 22
171 scrapped: 23
171 scrapped: 24
171 scrapped: 25
171 scrapped: 26
171 scrapped: 27
171 scrapped: 28
171 scrapped: 29
171 scrapped: 30
171 scrapped: 31
171 scrapped: 32
171 scrapped: 33
171 scrapped: 34
171 scrapped: 35
171 scrapped: 36
171 scrapped: 37
171 scrapped: 38
171 scrapped: 39
171 scrapped: 40
171 scrapped: 41
171 scrapped: 42
171 scrapped: 43
171 scrapped: 44


In [ ]:
pd.set_option('display.max_columns', 34)
result.head()

In [ ]:
result.to_csv('thegioididong.csv',encoding='utf-8')